# Data Analysis using MySQL and pandas in Python

This script demonstrates an approach that leverages structured database scheme using SQL for the same data analysis task.
MySQL is chosen as the underlying DBMS.
The module `mysql.connector` provides users working with Python a convenient interaction with MySQL databases.

While in some steps, some preprocessing is done on the input pandas dataframe, the focus of this task is on create databases in SQL from a dataframe.

## Setup

In [1]:
import pandas as pd
import mysql.connector

In [2]:
df = pd.read_csv('./electronics.csv')

In [3]:
df

,item_id,user_id,rating,timestamp,model_attr,category,brand,year,user_attr,split
0,0,0,5.0,1999-06-13,Female,Portable Audio & Video,NaN,1999,NaN,0
1,0,1,5.0,1999-06-14,Female,Portable Audio & Video,NaN,1999,NaN,0
2,0,2,3.0,1999-06-17,Female,Portable Audio & Video,NaN,1999,NaN,0
3,0,3,1.0,1999-07-01,Female,Portable Audio & Video,NaN,1999,NaN,0
4,0,4,2.0,1999-07-06,Female,Portable Audio & Video,NaN,1999,NaN,0
...,...,...,...,...,...,...,...,...,...,...
1292949,9478,1157628,1.0,2018-09-26,Female,Headphones,Etre Jeune,2017,NaN,0
1292950,9435,1157629,5.0,2018-09-26,Female,Computers & Accessories,NaN,2017,NaN,0
1292951,9305,1157630,3.0,2018-09-26,Female,Computers & Accessories,NaN,2016,NaN,0
1292952,9303,1157631,5.0,2018-09-29,Male,Headphones,NaN,2018,NaN,0


**Notes:** replace with real username and password to run this command

In [4]:
db = mysql.connector.connect(
	host='localhost',
	user='username',
	password='password'
)

In [6]:
cursor = db.cursor(buffered=True) # buffered cursor!

In [7]:
# create database Portofolios if it does not exist
cursor.execute("create database if not exists electronics")
db.commit()

In [8]:
cursor.execute('show databases')

for x in cursor:
    print(x)

('Portfolios',)
('electronics',)
('information_schema',)
('mysql',)
('performance_schema',)
('sys',)


In [9]:
cursor.execute("use electronics")

In [10]:
command = 'DROP TABLES ratings'
cursor.execute(command)

command = 'DROP TABLES items'
cursor.execute(command)

command = 'DROP TABLES users'
cursor.execute(command)

## Create Tables

The provided CSV data is stored in MySQL using the following SQL schema consisting of three tables: `users`, `items`, and `ratings`.

**Notes:** for the most users, the `user_attr` column is NaN. Otherwise, `user_attr` is either Famale or Male.

In [11]:
cursor.execute("CREATE TABLE users (user_id INT PRIMARY KEY, user_attr VARCHAR(50))")

In [12]:
cursor.execute("CREATE TABLE items (item_id INT PRIMARY KEY, model_attr VARCHAR(50), category VARCHAR(100), brand VARCHAR(100), year INT)")

This database scheme allows one user who bought the same item more than once can rate the same item multiple times.

In [13]:
cursor.execute("CREATE TABLE ratings (rating_id SERIAL PRIMARY KEY, \
    user_id INT, \
    item_id INT, \
    rating DECIMAL(2, 1), \
    timestamp DATE, \
    FOREIGN KEY (user_id) REFERENCES users(user_id), \
    FOREIGN KEY (item_id) REFERENCES items(item_id))")

In [14]:
cursor.execute('show tables')

for x in cursor:
    print(x)

('items',)
('ratings',)
('users',)


## Insert Data into Tables

### Table `users`

**Notes:** In input data, one user may buy many products or the same product many times. 
Thus, the input data stores `user_id` with other attributes belong to users redundantly. 
To avoid the redundancy, `drop_duplicates` is called on the input `df` to eliminate such duplicates which is followed by insertions to the table `users`. 

In [15]:
tmp_df = df.drop_duplicates(['user_id','user_attr'])[['user_id','user_attr']]

In [16]:
tmp_df['user_attr'] = tmp_df.user_attr.astype(str)

In [17]:
users_entries = [list(a) for a in zip(tmp_df['user_id'], tmp_df['user_attr'])]

In [18]:
command = "INSERT INTO users (user_id, user_attr) VALUES (%s, %s)"
cursor.executemany(command, users_entries)

**Notes:** Nan values on the `user_attr` column is converted to NULL which is a well-defined representation of an empty field in SQL.

In [19]:
command = "UPDATE users SET user_attr = NULL WHERE user_attr = 'nan'"
cursor.execute(command)

**Notes:** Length of tables `users`

In [20]:
command = "SELECT COUNT(*) FROM users"
cursor.execute(command)

for x in cursor:
    print(x)

(1157633,)


In [21]:
command = "SELECT * FROM users"
cursor.execute(command)

**Notes:** Print first 5 rows of the newly created table `users`

In [22]:
result = cursor.fetchmany(5)
for x in result:
    print(x)

(0, None)
(1, None)
(2, None)
(3, None)
(4, None)


### Table `items`

In [23]:
df.brand = df.brand.astype(str)

In [24]:
tmp_df = df.drop_duplicates(['item_id'])[['item_id', 'model_attr', 'category', 'brand', 'year']]

In [25]:
items_entries = [list(a) for a in zip(tmp_df['item_id'], tmp_df['model_attr'], tmp_df['category'], tmp_df['brand'], tmp_df['year'])]

In [26]:
command = "INSERT INTO items (item_id, model_attr, category, brand, year) VALUES (%s, %s, %s, %s, %s)"
cursor.executemany(command, items_entries)

In [27]:
command = "UPDATE items SET brand = NULL WHERE brand = 'nan'"
cursor.execute(command)

**Notes:** Length of tables `items`

In [28]:
command = "SELECT COUNT(*) FROM items"
cursor.execute(command)

for x in cursor:
    print(x)

(9560,)


In [29]:
command = "SELECT * FROM items"
cursor.execute(command)

**Notes:** Print first 5 rows of the newly created table `items`

In [30]:
result = cursor.fetchmany(5)
for x in result:
    print(x)

(0, 'Female', 'Portable Audio & Video', None, 1999)
(1, 'Female', 'Computers & Accessories', 'HP', 2000)
(2, 'Female&Male', 'Headphones', None, 2000)
(3, 'Female', 'Camera & Photo', None, 2005)
(4, 'Female', 'Television & Video', 'Philips', 2000)


### Table `ratings`

In [31]:
ratings_entries = [list(a) for a in zip(df['user_id'], tmp_df['item_id'], df['rating'], df['timestamp'])]

In [32]:
command = "INSERT INTO ratings (user_id, item_id, rating, timestamp) VALUES (%s, %s, %s, %s)"
cursor.executemany(command, ratings_entries)

**Notes:** Length of tables `ratings`

In [33]:
command = "SELECT COUNT(*) FROM ratings"
cursor.execute(command)

for x in cursor:
    print(x)

(9560,)


In [34]:
command = "SELECT * FROM ratings"
cursor.execute(command)

**Notes:** Print first 5 rows of the newly created table `ratings`

In [35]:
result = cursor.fetchmany(5)
for x in result:
    print(x)

(1, 0, 0, Decimal('5.0'), datetime.date(1999, 6, 13))
(2, 1, 1, Decimal('5.0'), datetime.date(1999, 6, 14))
(3, 2, 2, Decimal('3.0'), datetime.date(1999, 6, 17))
(4, 3, 3, Decimal('1.0'), datetime.date(1999, 7, 1))
(5, 4, 4, Decimal('2.0'), datetime.date(1999, 7, 6))


## Close DB Connection

In [36]:
cursor.close()
db.close()

## Conclusion

In this example I demonstrated how to create tables in SQL from input data given in CSV.
The database scheme is designed to eliminate the redundancy found in the original input data.
The approach for doing data analysis using SQL may result in queries that are easier to write and more understandable than the approach using only `pandas dataframe` as I also demonstrated in the other script in this project.